<a href="https://colab.research.google.com/github/Feranie/Hierarchical-Classification-Project/blob/main/teste_wilcoxon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np  # Importa a biblioteca NumPy para operações numéricas e arrays
import pandas as pd  # Importa a biblioteca Pandas para manipulação de dados e DataFrames
from scipy.stats import wilcoxon  # Importa o teste de Wilcoxon da biblioteca SciPy para testes estatísticos não-paramétricos
from statsmodels.stats.multitest import multipletests  # Importa função para correção de múltiplos testes

# --- Dados experimentais ---
# Dicionário contendo os conjuntos de dados experimentais organizados por dataset e método
datasets = {
    'GPCR-Pfam': {
        'baseline': [57.72, 58.53, 60.42, 58.44, 59.27, 60.41, 59.61, 60.92, 60.29, 58.59],  # Resultados do método baseline
        'hierarchical': [58.43, 60.1, 60.64, 58.43, 60.08, 61.28, 59.62, 61.37, 60.3, 60.92],  # Resultados do método hierárquico
        'flat_ir': [23.41, 24.35, 22.92, 28.76, 29.24, 24.34, 30.43, 24.07, 26.14, 26.71]  # Resultados do método IR plano
    },
    'GPCR-Prints': {
        'baseline': [74.74, 75.44, 76.7, 75.74, 72.3, 75.75, 73.65, 76.48, 79.52, 74.52],
        'hierarchical': [74.92, 76.09, 77.06, 76.25, 74.61, 76.39, 79.91, 77.32, 80.87, 80.88],
        'flat_ir': [29.12, 30.61, 32.26, 30.5, 31.89, 30.55, 25.63, 32.39, 31.55, 32.33]
    },
    'GPCR-Prosite': {
        'baseline': [56.5, 58.81, 57.67, 56.07, 55.23, 57.3, 59.82, 59.68, 57.54, 61.19],
        'hierarchical': [57.19, 60.25, 57.52, 56.44, 57.99, 58.66, 60.96, 61.72, 59.06, 62.54],
        'flat_ir': [24.1, 25.81, 26.27, 25.29, 25.58, 24.42, 26.43, 26.87, 24.22, 26.99]
    },
    'GPCR-Interpro': {
        'baseline': [76.47, 79.94, 76.69, 76.97, 76.05, 76.22, 77.63, 75.45, 77.42, 78.34],
        'hierarchical': [74.8, 78.82, 74.47, 76.55, 79.15, 78.91, 78.02, 77.91, 80.93, 81.82],
        'flat_ir': [26.41, 25.62, 25.57, 22.66, 24.59, 26.92, 25.75, 27.98, 26.84, 28.93]
    },
    'EC-Pfam': {
        'baseline': [90.35, 90.19, 90.19, 91.51, 89.62, 91.00, 89.82, 89.75, 89.78, 89.66],
        'hierarchical': [76.56, 81.7, 78.14, 81.29, 79.63, 77.87, 81.68, 78.96, 81.78, 79.82],
        'flat_ir': [14.39, 7.39, 10.8, 13.42, 11.15, 17.73, 11.99, 20.13, 9.94, 15.07]
    },
    'EC-Prints': {
        'baseline': [92.53, 93.78, 92.87, 92.71, 91.89, 91.27, 91.32, 92.51, 92.16, 92.41],
        'hierarchical': [82.32, 80.7, 81.61, 80.51, 81.93, 77.65, 81.44, 79.55, 83.19, 81.07],
        'flat_ir': [7.21, 11.54, 8.24, 6.69, 7.3, 9.5, 8.31, 8.85, 6.75, 4.11]
    },
    'EC-Prosite': {
        'baseline': [93.7, 92.92, 94.17, 93.53, 94.76, 94.33, 92.52, 93.44, 93.34, 93.77],
        'hierarchical': [86.29, 84.3, 87.28, 85.63, 84.96, 86.48, 87.17, 86.85, 88.62, 85.28],
        'flat_ir': [7.2, 8.35, 7.69, 7.21, 8.88, 7.95, 9.08, 8.94, 9.67, 8.9]
    },
    'EC-Interpro': {
        'baseline': [89.98, 92.43, 91.2, 90.68, 91.44, 91.09, 92.24, 90.91, 92.07, 90.28],
        'hierarchical': [79.74, 80.56, 84.81, 82.18, 79.86, 79.42, 79.64, 79.12, 83.05, 80.63],
        'flat_ir': [8.19, 9.26, 8.86, 9.61, 10.98, 7.24, 10.74, 9.62, 9.25, 8.03]
    }
}

# Dicionário contendo as estatísticas descritivas (média e desvio padrão) pré-calculadas para cada dataset e método
dataset_stats = {
    'GPCR-Pfam': {'baseline_mean': 59.42, 'baseline_std': 1.07, 'hierarchical_mean': 60.12, 'hierarchical_std': 1.04, 'flat_ir_mean': 26.04, 'flat_ir_std': 2.66},
    'GPCR-Prints': {'baseline_mean': 75.48, 'baseline_std': 1.94, 'hierarchical_mean': 77.43, 'hierarchical_std': 2.32, 'flat_ir_mean': 30.68, 'flat_ir_std': 2.06},
    'GPCR-Prosite': {'baseline_mean': 57.98, 'baseline_std': 1.87, 'hierarchical_mean': 59.23, 'hierarchical_std': 2.05, 'flat_ir_mean': 25.60, 'flat_ir_std': 1.07},
    'GPCR-Interpro': {'baseline_mean': 77.12, 'baseline_std': 1.30, 'hierarchical_mean': 78.14, 'hierarchical_std': 2.37, 'flat_ir_mean': 26.13, 'flat_ir_std': 1.75},
    'EC-Pfam': {'baseline_mean': 90.19, 'baseline_std': 0.63, 'hierarchical_mean': 79.74, 'hierarchical_std': 1.85, 'flat_ir_mean': 13.20, 'flat_ir_std': 3.79},
    'EC-Prints': {'baseline_mean': 92.35, 'baseline_std': 0.74, 'hierarchical_mean': 81.00, 'hierarchical_std': 1.55, 'flat_ir_mean': 7.85, 'flat_ir_std': 1.97},
    'EC-Prosite': {'baseline_mean': 93.65, 'baseline_std': 0.66, 'hierarchical_mean': 86.29, 'hierarchical_std': 1.28, 'flat_ir_mean': 8.39, 'flat_ir_std': 0.84},
    'EC-Interpro': {'baseline_mean': 91.23, 'baseline_std': 0.82, 'hierarchical_mean': 80.90, 'hierarchical_std': 1.86, 'flat_ir_mean': 9.18, 'flat_ir_std': 1.17}
}

def perform_wilcoxon_tests_with_multipletests(datasets, dataset_stats):
    """
    Função que realiza testes de Wilcoxon para todas as comparações e aplica correção de Bonferroni
    """
    results = []  # Lista para armazenar todos os resultados dos testes
    pvals = []  # Lista para armazenar todos os p-valores para correção múltipla
    comparison_info = []  # Lista para armazenar informações detalhadas das comparações

    # Loop através de cada dataset para realizar as comparações
    for dataset_name in datasets.keys():
        # Converte os dados para arrays NumPy para facilitar os cálculos estatísticos
        b = np.array(datasets[dataset_name]['baseline'])  # Array dos valores baseline
        h = np.array(datasets[dataset_name]['hierarchical'])  # Array dos valores hierárquicos
        f = np.array(datasets[dataset_name]['flat_ir'])  # Array dos valores IR plano

        stats = dataset_stats[dataset_name]  # Obtém as estatísticas pré-calculadas do dataset atual

        # Realiza os três testes de Wilcoxon: baseline vs hierarchical, baseline vs flat_ir, hierarchical vs flat_ir
        stat1, p1 = wilcoxon(b, h)  # Teste de Wilcoxon entre baseline e hierarchical
        stat2, p2 = wilcoxon(b, f)  # Teste de Wilcoxon entre baseline e flat_ir
        stat3, p3 = wilcoxon(h, f)  # Teste de Wilcoxon entre hierarchical e flat_ir

        pvals += [p1, p2, p3]  # Adiciona os p-valores à lista para correção múltipla

        # Cria dicionários com informações detalhadas de cada comparação
        comparison_info += [
            dict(Dataset=dataset_name, Comparison='Baseline vs IRH', Statistic=stat1, P_value=p1, Effect='IRH > Baseline' if stats['hierarchical_mean'] > stats['baseline_mean'] else 'Baseline > IRH'),
            dict(Dataset=dataset_name, Comparison='Baseline vs IR', Statistic=stat2, P_value=p2, Effect='Baseline > IR' if stats['baseline_mean'] > stats['flat_ir_mean'] else 'IR > Baseline'),
            dict(Dataset=dataset_name, Comparison='IRH vs IR', Statistic=stat3, P_value=p3, Effect='IRH > IR' if stats['hierarchical_mean'] > stats['flat_ir_mean'] else 'IR > IRH')
        ]

    # Aplicação da correção de Bonferroni para múltiplos testes com alpha = 0.05
    reject, pval_adj, _, _ = multipletests(pvals, alpha=0.05, method='bonferroni')

    # Adiciona os p-valores ajustados e significância às informações das comparações
    for i, info in enumerate(comparison_info):
        info['P_value_adjusted'] = pval_adj[i]  # P-valor ajustado pela correção de Bonferroni
        info['Significant_adjusted'] = reject[i]  # Boolean indicando se é significativo após correção
        results.append(info)  # Adiciona à lista de resultados

    return pd.DataFrame(results)  # Retorna um DataFrame com todos os resultados

def create_results_table_formatted(results_df, dataset_stats):
    """
    Função que cria e exibe uma tabela formatada com os resultados dos testes estatísticos
    """
    table_data = []  # Lista para armazenar os dados da tabela

    # Loop através de cada dataset para construir as linhas da tabela
    for dataset_name, stats in dataset_stats.items():
        ds_res = results_df[results_df['Dataset'] == dataset_name]  # Filtra resultados do dataset atual

        # Função interna para obter símbolo de significância estatística
        def get_star(cmp):
            sig = ds_res[ds_res['Comparison'] == cmp]['Significant_adjusted'].iloc[0]  # Verifica se é significativo
            return '★' if sig else '—'  # Retorna estrela se significativo, traço se não

        # Constrói linha da tabela com médias, desvios padrão e símbolos de significância
        table_data.append({
            'Bases de dados': dataset_name,
            'hF (Sem Seleção) (desvio padrão)': f"{stats['baseline_mean']:.2f} ({stats['baseline_std']:.2f})",
            'hF (IRH) (desvio padrão)': f"{stats['hierarchical_mean']:.2f} ({stats['hierarchical_std']:.2f})",
            'hF (IR) (desvio padrão)': f"{stats['flat_ir_mean']:.2f} ({stats['flat_ir_std']:.2f})",
            'Baseline vs IRH': get_star('Baseline vs IRH'),
            'IRH vs IR': get_star('IRH vs IR'),
            'Baseline vs IR': get_star('Baseline vs IR'),
        })

    df_table = pd.DataFrame(table_data)  # Cria DataFrame com os dados da tabela

    # Contagem de "vitórias" (testes significativos) para cada tipo de comparação
    win_irh = df_table['Baseline vs IRH'].tolist().count('★')  # Conta quantas vezes IRH foi significativamente melhor que Baseline
    win_ir = df_table['Baseline vs IR'].tolist().count('★')  # Conta quantas vezes IR foi significativamente diferente de Baseline
    win_irh_ir = df_table['IRH vs IR'].tolist().count('★')  # Conta quantas vezes IRH foi significativamente melhor que IR

    # Exibição da tabela formatada com bordas ASCII
    print("┌────────────────────┬──────────────────────────────┬────────────────────────────┬────────────────────────────┬───────────────┬───────────────┬───────────────┐")
    print("│ Bases de dados     │ hF (Sem Seleção)             │ hF (IRH)                   │ hF (IR)                    │ Baseline vs   │ IRH vs IR    │ Baseline vs   │")
    print("│                    │ (desvio padrão)              │ (desvio padrão)            │ (desvio padrão)            │ IRH           │              │ IR            │")
    print("├────────────────────┼──────────────────────────────┼────────────────────────────┼────────────────────────────┼───────────────┼───────────────┼───────────────┤")

    # Loop para imprimir cada linha da tabela
    for _, row in df_table.iterrows():
        print(f"│ {row['Bases de dados']:<19} │ {row['hF (Sem Seleção) (desvio padrão)']:<26} │ {row['hF (IRH) (desvio padrão)']:<26} │ {row['hF (IR) (desvio padrão)']:<26} │ {row['Baseline vs IRH']:^13} │ {row['IRH vs IR']:^13} │ {row['Baseline vs IR']:^13} │")

    # Linha separadora e linha de contagem de vitórias
    print("├────────────────────┴──────────────────────────────┴────────────────────────────┴────────────────────────────┴───────────────┴───────────────┴───────────────┤")
    print(f"│ {'Vitórias (★)':<19} {'':<26} {'':<26} {'':<26} │ {win_irh:^13} │ {win_irh_ir:^13} │ {win_ir:^13} │")
    print("└────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘")

    # Legenda explicativa
    print("\nLegenda: ★ = significativo após correção Bonferroni ; — = não significativo")

    return df_table  # Retorna o DataFrame da tabela

# Bloco principal de execução do programa
if __name__ == "__main__":
    print("🚀 INÍCIO DA ANÁLISE ESTATÍSTICA")  # Mensagem de início da análise
    results_df = perform_wilcoxon_tests_with_multipletests(datasets, dataset_stats)  # Executa os testes estatísticos
    create_results_table_formatted(results_df, dataset_stats)  # Cria e exibe a tabela formatada com os resultados

🚀 INÍCIO DA ANÁLISE ESTATÍSTICA
┌────────────────────┬──────────────────────────────┬────────────────────────────┬────────────────────────────┬───────────────┬───────────────┬───────────────┐
│ Bases de dados     │ hF (Sem Seleção)             │ hF (IRH)                   │ hF (IR)                    │ Baseline vs   │ IRH vs IR    │ Baseline vs   │
│                    │ (desvio padrão)              │ (desvio padrão)            │ (desvio padrão)            │ IRH           │              │ IR            │
├────────────────────┼──────────────────────────────┼────────────────────────────┼────────────────────────────┼───────────────┼───────────────┼───────────────┤
│ GPCR-Pfam           │ 59.42 (1.07)               │ 60.12 (1.04)               │ 26.04 (2.66)               │       —       │       ★       │       ★       │
│ GPCR-Prints         │ 75.48 (1.94)               │ 77.43 (2.32)               │ 30.68 (2.06)               │       ★       │       ★       │       ★       │
│ GPCR-Prosi